2. Employee Record Management System
 Objective:
  Build an employee record management application.
   Requirements:
- Create a system to manage employee records. 
- Allow adding, updating, and deleting employee records.
- Implement search functionality based on different criteria (name, department, etc.). 
- Display reporting hierarchy of employees.
- Handle edge cases such as circular references in the reporting structure.

In [6]:
class Employee:
    def __init__(self, id, name, department, manager_id=None):
        self.id = id
        self.name = name
        self.department = department
        self.manager_id = manager_id

    def __repr__(self):
        return f"Employee({self.id}, {self.name}, {self.department}, {self.manager_id})"


class EmployeeManagementSystem:
    def __init__(self):
        self.employees = {}

    def add_employee(self, id, name, department, manager_id=None):
        if id in self.employees:
            raise ValueError(f"Employee with id {id} already exists.")
        if manager_id and manager_id not in self.employees:
            raise ValueError(f"Manager with id {manager_id} does not exist.")
        self.employees[id] = Employee(id, name, department, manager_id)

    def update_employee(self, id, name=None, department=None, manager_id=None):
        if id not in self.employees:
            raise ValueError(f"Employee with id {id} does not exist.")
        if manager_id and manager_id not in self.employees:
            raise ValueError(f"Manager with id {manager_id} does not exist.")
        employee = self.employees[id]
        if name:
            employee.name = name
        if department:
            employee.department = department
        if manager_id:
            # Temporarily update to check for circular reference
            old_manager_id = employee.manager_id
            employee.manager_id = manager_id
            try:
                self.check_circular_references()
            except ValueError:
                # Revert if circular reference detected
                employee.manager_id = old_manager_id
                raise

    def delete_employee(self, id):
        if id not in self.employees:
            raise ValueError(f"Employee with id {id} does not exist.")
        # Also need to handle cascading delete for subordinates
        self._delete_subordinates(id)
        del self.employees[id]

    def _delete_subordinates(self, manager_id):
        subordinates = [emp_id for emp_id, emp in self.employees.items() if emp.manager_id == manager_id]
        for sub_id in subordinates:
            self.delete_employee(sub_id)

    def search_employee(self, **criteria):
        results = self.employees.values()
        for key, value in criteria.items():
            results = [emp for emp in results if getattr(emp, key) == value]
        return results

    def display_hierarchy(self):
        def display(emp_id, level=0):
            print('  ' * level + str(self.employees[emp_id]))
            subordinates = [emp_id for emp_id, emp in self.employees.items() if emp.manager_id == emp_id]
            for sub_id in subordinates:
                display(sub_id, level + 1)

        top_level_managers = [emp_id for emp_id, emp in self.employees.items() if emp.manager_id is None]
        for manager_id in top_level_managers:
            display(manager_id)

    def check_circular_references(self):
        def visit(emp_id, visited):
            if emp_id in visited:
                raise ValueError(f"Circular reference detected involving employee {emp_id}")
            visited.add(emp_id)
            subordinates = [emp_id for emp_id, emp in self.employees.items() if emp.manager_id == emp_id]
            for sub_id in subordinates:
                visit(sub_id, visited)
            visited.remove(emp_id)

        for emp_id in self.employees:
            visit(emp_id, set())

# Example usage
ems = EmployeeManagementSystem()
ems.add_employee(1, 'Alice', 'Engineering')
ems.add_employee(2, 'Bob', 'HR', 1)
ems.add_employee(3, 'Charlie', 'Engineering', 1)
ems.add_employee(4, 'David', 'Engineering', 3)

print("All employees:")
for emp in ems.employees.values():
    print(emp)

print("\nSearching for employees in Engineering:")
print(ems.search_employee(department='Engineering'))

print("\nHierarchy:")
ems.display_hierarchy()

# add a circular reference to test
try:
    ems.update_employee(1, manager_id=4)
except ValueError as e:
    print(f"\nError: {e}")

# Check for circular references
try:
    ems.check_circular_references()
except ValueError as e:
    print(f"\nError: {e}")

# Update hierarchy 
try:
    ems.update_employee(2, manager_id=3)  # Bob now reports to Charlie
except ValueError as e:
    print(f"\nError: {e}")

print("\nUpdated hierarchy:" , ems.display_hierarchy())


All employees:
Employee(1, Alice, Engineering, None)
Employee(2, Bob, HR, 1)
Employee(3, Charlie, Engineering, 1)
Employee(4, David, Engineering, 3)

Searching for employees in Engineering:
[Employee(1, Alice, Engineering, None), Employee(3, Charlie, Engineering, 1), Employee(4, David, Engineering, 3)]

Hierarchy:
Employee(1, Alice, Engineering, None)

Updated hierarchy: None
